In [31]:
%matplotlib inline

print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
import re
import json
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
    nltk.download("words")

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
#from mpl_toolkits.mplot3d import Axes3D
#from mpl_toolkits.mplot3d import proj3d
import csv

with open('emoji/emoji_final_join.json',encoding='utf-8') as f: 
    emoji_list = json.load(f)



def clean(text,emoji):
    text = text.lower()
    text = text.split()
    print(text)
    newtext = [x for x in text if not x.startswith('@') and not x.startswith('https')]
    newtext = ' '.join(newtext)
    tokenizer = RegexpTokenizer(r'\w+')
    newtext = tokenizer.tokenize(newtext)
    #stop_words = stopwords.words('english')
    #newtext =  [w for w in newtext if not w in stop_words]
    #stopwords.words will delete word 'not' which is important for sentiment analysis, so we don't remove stopwords.
    stemmer = SnowballStemmer('english')
    newtext = list(map(stemmer.stem, newtext))
    emojis = split(emoji)
    newtext = newtext + emojis
    newtext = ' '.join(newtext)
    
    return newtext
    



with open('smalldata.csv',encoding="utf8") as words_file:
    csv_reader = csv.DictReader(words_file, delimiter = ',')
    data = []
    for row in csv_reader:
        cleaned_row = []
        clean_text = clean(row['Text'],row['Emoji'])
        cleaned_row.append(clean_text)
        if float(row['Sentiment']) > 0:
            cleaned_row.append('positive')
        elif float(row['Sentiment']) < 0:
            cleaned_row.append('negative')
        else:
            if float(row['Emoji_sentiment']) > 0:
                cleaned_row.append('positive')
            elif float(row['Emoji_sentiment']) < 0:
                cleaned_row.append('negative')
            else:
                cleaned_row.append('neutral')
        
        cleaned_row.append(row['Emoji'])
        data.append(np.array(cleaned_row))
    data = np.array(data)
    
print(data[:10])
#Y = data[:,0]
#X = np.row_stack((data[:,1:4].astype(np.float)))


Automatically created module for IPython interactive environment
['i’m', 'realized', 'tonight', 'i', 'was', 'a', 'titty', 'man', '😎😂']


NameError: name 'split' is not defined

In [29]:
a= ['a','m']
b = ['sss','sss']
a+b

['a', 'm', 'sss', 'sss']